In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting, datasets, image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# =============================================================================
# 1. File paths
# =============================================================================
thresholded_task_effect_img_path = "/path/to/stress_effect_thresholded_tvals_vox.nii"
bayes_img_path = "/path/to/BFlog_ROI.nii"
roi_atlas_path = "/path/to/BN246_atlas.nii.gz"
output_path = "/path/to/output/BFlog_filtered_ROIs.png"
filtered_bayes_output_path = "/path/to/output/BFlog_filtered_ROIs.nii.gz"

# =============================================================================
# 2. Function Definitions
# =============================================================================

def calculate_roi_significance(atlas_data, significant_voxels, threshold):
    """
    Return a list of ROI IDs from atlas_data that meet or exceed a given
    threshold for proportion of significant_voxels.
    """
    roi_ids = np.unique(atlas_data)[1:]  # Exclude background (0)
    significant_rois = []
    for roi_id in roi_ids:
        roi_mask = (atlas_data == roi_id)
        proportion = np.sum(significant_voxels[roi_mask]) / np.sum(roi_mask)
        if proportion >= threshold:
            significant_rois.append(roi_id)
    return significant_rois

def recode_bayes_factors(bf_data, categories):
    """
    Recode Bayes factor data into integer-coded categories.
    
    Parameters
    ----------
    bf_data : np.ndarray
        Array of Bayes factor values (can contain NaNs).
    categories : list of tuples
        List of (low, high, value), where `value` is the integer code
        to assign if bf_data is in (low, high].

    Returns
    -------
    recoded_data : np.ndarray
        Same shape as bf_data, with integer codes (or NaN if no category applies).
    """
    recoded_data = np.full(bf_data.shape, np.nan)
    for (low, high, value) in categories:
        mask = (bf_data > low) & (bf_data <= high)
        recoded_data[mask] = value
    return recoded_data

def plot_filtered_bayes_factors(
    img,
    standard_brain,
    cmap,
    legend_info,
    title,
    output_path
):
    coords_display_modes = [
        ([-48], 'x'),
        ([16],  'y'),
        ([3],   'z'),
        ([6],   'x'),
        ([-5],  'y'),
        ([33],  'z'),
    ]

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.patch.set_facecolor('white')
    fig.suptitle(title, fontsize=20)
    axes = axes.flatten()

    positions = [
        [0.05, 0.55, 0.25, 0.35],
        [0.35, 0.55, 0.25, 0.35],
        [0.65, 0.55, 0.25, 0.35],
        [0.05, 0.1,  0.25, 0.35],
        [0.35, 0.1,  0.25, 0.35],
        [0.65, 0.1,  0.25, 0.35]
    ]

    for i, (cut_coords, display_mode) in enumerate(coords_display_modes):
        axes[i].set_position(positions[i])
        plotting.plot_stat_map(
            img,                     # The masked recoded BF image
            bg_img=standard_brain,   # Show MNI background in grayscale
            cut_coords=cut_coords,
            display_mode=display_mode,
            colorbar=False,          # We'll use a manual legend
            axes=axes[i],
            draw_cross=False,
            cmap=cmap,
            threshold=None,          # No thresholding - show all categories
            vmin=1,                  # Lowest discrete label
            vmax=10,                 # Highest discrete label
            interpolation='nearest', # Avoid smoothing across discrete labels
            dim=0.5,                   # Turn off background dimming (no blue tint)
            alpha=0.85,               # Overlay is fully opaque
            black_bg=False
        )
        axes[i].tick_params(axis='both', which='major', labelsize=14)

    # Manual legend
    patches = [
        mpatches.Patch(color=color, label=label)
        for (color, label) in legend_info
    ]
    fig.legend(
        handles=patches,
        title="Evidence Category",
        loc='center left',
        bbox_to_anchor=(1.02, 0.5),
        frameon=False,
        fontsize=20,
        title_fontsize=20
    )

    plt.subplots_adjust(right=0.85)
    plt.savefig(output_path, bbox_inches='tight')
    plt.show()


# =============================================================================
# 3. Load Images
# =============================================================================
thresholded_task_effect_img = image.load_img(thresholded_task_effect_img_path)
bayes_img = image.load_img(bayes_img_path)

# Resample ROI atlas to match the thresholded task effect image
roi_atlas = image.resample_to_img(
    image.load_img(roi_atlas_path),
    thresholded_task_effect_img,
    interpolation="nearest"
)

standard_brain = datasets.load_mni152_template()

# =============================================================================
# 4. Extract Data Arrays
# =============================================================================
thresholded_task_effect_data = np.squeeze(thresholded_task_effect_img.get_fdata())
roi_atlas_data = np.squeeze(roi_atlas.get_fdata())
bayes_data = np.squeeze(bayes_img.get_fdata())

# =============================================================================
# 5. Identify Significant Voxels
# =============================================================================
# Here, we assume "significant" is simply thresholded > 0 because the image has
# already been recoded to zero in case of p > 0.05
significant_voxels = thresholded_task_effect_data != 0

# =============================================================================
# 6. Calculate Proportion of Significant Voxels per ROI
# =============================================================================
roi_ids = np.unique(roi_atlas_data)[1:]  # Exclude background (0)
proportions = []
for roi_id in roi_ids:
    roi_mask = (roi_atlas_data == roi_id)
    prop = np.sum(significant_voxels[roi_mask]) / np.sum(roi_mask)
    proportions.append(prop)

# 75th percentile threshold for "significant" ROIs
threshold = np.percentile(proportions, 75)
print(f"75th Percentile Threshold: {threshold:.2f}")

# =============================================================================
# 7. Identify Significant ROIs
# =============================================================================
significant_rois = calculate_roi_significance(roi_atlas_data, significant_voxels, threshold)
print(f"Number of Task-Responsive ROIs: {len(significant_rois)}")

bf_positive_rois_clean = [int(roi) for roi in significant_rois]
print(f"ROI Numbers of Task-Responsive ROIs: {bf_positive_rois_clean}")

# =============================================================================
# 8. Bayesian Factor Analysis (Within Significant ROIs)
# =============================================================================
num_evidence_H1 = 0  # Count of significant ROIs with BF > 0
num_evidence_H0 = 0  # Count of significant ROIs with BF < 0

for roi_id in significant_rois:
    roi_mask = (roi_atlas_data == roi_id)
    mean_bf = np.nanmean(bayes_data[roi_mask])
    if mean_bf > 0:
        num_evidence_H1 += 1
    elif mean_bf < 0:
        num_evidence_H0 += 1

# Percentages
total_significant_rois = len(significant_rois)
percentage_evidence_H1 = (num_evidence_H1 / total_significant_rois) * 100
percentage_evidence_H0 = (num_evidence_H0 / total_significant_rois) * 100

print(f"Number of Task-Responsive ROIs with Evidence for Site Differences (BF > 0): "
      f"{num_evidence_H1} ({percentage_evidence_H1:.2f}%)")
print(f"Number of Task-Responsive ROIs with No Evidence for Site Differences (BF < 0): "
      f"{num_evidence_H0} ({percentage_evidence_H0:.2f}%)")

# Identify which ROIs have BF > 0
bf_positive_rois = []
for roi_id in significant_rois:
    roi_mask = (roi_atlas_data == roi_id)
    if np.any(bayes_data[roi_mask] > 0):
        bf_positive_rois.append(roi_id)

bf_positive_rois_clean = [int(roi) for roi in bf_positive_rois]
print(f"ROI Numbers with BF > 0: {bf_positive_rois_clean}")

# =============================================================================
# 9. Create Masked Bayes Data for Only Significant ROIs
# =============================================================================
filtered_bayes_data = np.full(bayes_data.shape, np.nan)
for roi_id in significant_rois:
    roi_mask = (roi_atlas_data == roi_id)
    filtered_bayes_data[roi_mask] = bayes_data[roi_mask]

# Save the filtered Bayes factor image (before recoding)
filtered_bayes_img = nib.Nifti1Image(filtered_bayes_data, bayes_img.affine)
nib.save(filtered_bayes_img, filtered_bayes_output_path)
print(f"Filtered Bayes factor image saved to: {filtered_bayes_output_path}")

# =============================================================================
# 10. Recode Filtered Bayes Data into Categories
# =============================================================================
# Define Bayes factor categories
categories = [
    (-np.inf, -4.61, 1),   # Extreme evidence for H0
    (-4.61,   -3.40, 2),   # Very strong evidence for H0
    (-3.40,   -2.30, 3),   # Strong evidence for H0
    (-2.30,   -1.10, 4),   # Moderate evidence for H0
    (-1.10,    0,    5),   # Anecdotal evidence for H0
    (0,        1.10, 6),   # Anecdotal evidence for H1
    (1.10,     2.30, 7),   # Moderate evidence for H1
    (2.30,     3.40, 8),   # Strong evidence for H1
    (3.40,     4.61, 9),   # Very strong evidence for H1
    (4.61,     np.inf, 10) # Extreme evidence for H1
]

filtered_bayes_data_recoded = recode_bayes_factors(filtered_bayes_data, categories)

# =============================================================================
# 12. Create Discrete Colormap and Legend Info
# =============================================================================

masked_data = np.ma.masked_invalid(filtered_bayes_data_recoded)
filtered_bayes_img_recoded = nib.Nifti1Image(masked_data, bayes_img.affine)

# Build discrete colormap for categories 1..10
color_map_dict = {
    1 : '#2166ac',  # Extreme H0
    2 : '#4d84bc',  # Very strong H0
    3 : '#79a3cd',  # Strong H0
    4 : '#a6c1dd',  # Moderate H0
    5 : '#d2e0ee',  # Anecdotal H0
    6 : '#efd0d4',  # Anecdotal H1
    7 : '#e0a2aa',  # Moderate H1
    8 : '#d0747f',  # Strong H1
    9 : '#c14655',  # Very strong H1
    10: '#b2182b'   # Extreme H1
}

# Find which integer values are actually in the data
non_nan_vals = filtered_bayes_data_recoded[~np.isnan(filtered_bayes_data_recoded)]
unique_values = np.unique(non_nan_vals).astype(int)

# Build color list in the order of present categories
corrected_colors = [color_map_dict[val] for val in unique_values]
cmap = ListedColormap(corrected_colors)

# Very important: set "bad" color to 'none', so masked voxels are transparent
cmap.set_bad(color='none')

# Build legend info (color + label)
legend_map_dict = {
    1 : "Extreme Evidence for H0",
    2 : "Very Strong Evidence for H0",
    3 : "Strong Evidence for H0",
    4 : "Moderate Evidence for H0",
    5 : "Anecdotal Evidence for H0",
    6 : "Anecdotal Evidence for H1",
    7 : "Moderate Evidence for H1",
    8 : "Strong Evidence for H1",
    9 : "Very Strong Evidence for H1",
    10: "Extreme Evidence for H1"
}
legend_labels = [legend_map_dict[val] for val in unique_values]
legend_info = list(zip(corrected_colors, legend_labels))

# =============================================================================
# 13. Plot the Recoded Bayesian Factor Image
# =============================================================================
plot_filtered_bayes_factors(
    img=filtered_bayes_img_recoded,
    standard_brain=standard_brain,
    cmap=cmap,
    legend_info=legend_info,
    title="Bayes Factors (Filtered by Task Effect ROIs)",
    output_path=output_path
)

In [ ]:
import numpy as np
import nibabel as nib
from nilearn import plotting, datasets
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

# =============================================================================
# 1. File paths & Setup
# =============================================================================
basepath = "/path/to/OBS_Bayes"

bf_ROI_path = f"{basepath}/BFlog_ROI.nii"
output_nonfiltered = f"{basepath}/BFlog_nonfiltered_ROIs.png"

# Load the standard MNI template as background
standard_brain = datasets.load_mni152_template()

# =============================================================================
# 2. Recode Function
# =============================================================================
def recode_bayes_factors(nifti_img):
    """
    Recode raw Bayes factor data into discrete categories 1..10,
    stored as a new Nifti image (NaNs remain masked).
    """
    data = nifti_img.get_fdata()
    recoded_data = np.full(data.shape, np.nan)  # Start with NaN for 'transparent'

    # Categories (low, high, code)
    categories = [
        (-np.inf, -4.61, 1),   # Extreme evidence for H0
        (-4.61,   -3.40, 2),   # Very strong evidence for H0
        (-3.40,   -2.30, 3),   # Strong evidence for H0
        (-2.30,   -1.10, 4),   # Moderate evidence for H0
        (-1.10,    0,    5),   # Anecdotal evidence for H0
        (0,        1.10, 6),   # Anecdotal evidence for H1
        (1.10,     2.30, 7),   # Moderate evidence for H1
        (2.30,     3.40, 8),   # Strong evidence for H1
        (3.40,     4.61, 9),   # Very strong evidence for H1
        (4.61,     np.inf, 10) # Extreme evidence for H1
    ]

    for (low, high, val) in categories:
        mask = (data > low) & (data <= high)
        recoded_data[mask] = val

    # Mask invalid (NaN) values so they remain transparent when plotting
    recoded_data = np.ma.masked_invalid(recoded_data)

    return nib.Nifti1Image(recoded_data, nifti_img.affine)

# =============================================================================
# 3. Discrete Colormap & Legend Configuration
# =============================================================================
# Predefined colors for each category 1..10
# (Same color order as your previous script.)
colors = [
    "#2166ac",  # 1 : Extreme H0
    "#4d84bc",  # 2 : Very strong H0
    "#79a3cd",  # 3 : Strong H0
    "#a6c1dd",  # 4 : Moderate H0
    "#d2e0ee",  # 5 : Anecdotal H0
    "#efd0d4",  # 6 : Anecdotal H1
    "#e0a2aa",  # 7 : Moderate H1
    "#d0747f",  # 8 : Strong H1
    "#c14655",  # 9 : Very strong H1
    "#b2182b"   # 10: Extreme H1
]

cmap = ListedColormap(colors)
cmap.set_bad(color="none")  # Transparent for any masked voxels

# For the legend
category_labels = [
    "Extreme Evidence for H0",
    "Very Strong Evidence for H0",
    "Strong Evidence for H0",
    "Moderate Evidence for H0",
    "Anecdotal Evidence for H0",
    "Anecdotal Evidence for H1",
    "Moderate Evidence for H1",
    "Strong Evidence for H1",
    "Very Strong Evidence for H1",
    "Extreme Evidence for H1"
]

# =============================================================================
# 4. Plotting Function
# =============================================================================
def plot_brain_slices(nifti_img, title, cmap, save_path):
    """
    Plots multiple slices (sagittal, coronal, axial) using nilearn's plot_stat_map,
    with the background set to the MNI template and a custom colormap
    for discrete Bayes factor categories.
    """
    # Slicing configuration
    coords_display_modes = [
        ([-48], "x"),
        ([16],  "y"),
        ([3],   "z"),
        ([6],   "x"),
        ([-5],  "y"),
        ([33],  "z")
    ]

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.patch.set_facecolor("white")
    fig.suptitle(title, fontsize=20)
    axes = axes.flatten()

    # Positions to avoid overlap with the legend
    positions = [
        [0.05, 0.55, 0.25, 0.35],
        [0.35, 0.55, 0.25, 0.35],
        [0.65, 0.55, 0.25, 0.35],
        [0.05, 0.1,  0.25, 0.35],
        [0.35, 0.1,  0.25, 0.35],
        [0.65, 0.1,  0.25, 0.35]
    ]

    # Plot each slice
    for i, (cut_coords, display_mode) in enumerate(coords_display_modes):
        axes[i].set_position(positions[i])
        plotting.plot_stat_map(
            nifti_img,
            bg_img=standard_brain,
            cut_coords=cut_coords,
            display_mode=display_mode,
            colorbar=False,         # We'll add our own legend
            axes=axes[i],
            draw_cross=False,
            cmap=cmap,
            threshold=None,         # Show all categories
            vmin=1,                 # Lowest integer category
            vmax=10,                # Highest integer category
            interpolation="nearest",
            dim=0.5,                # Slightly dim background for contrast
            alpha=0.85,             # Semi-transparent overlay
            black_bg=False
        )
        axes[i].tick_params(axis="both", which="major", labelsize=14)

    # Create custom legend
    patches = [
        mpatches.Patch(color=colors[i], label=category_labels[i])
        for i in range(len(colors))
    ]
    fig.legend(
        handles=patches,
        title="Evidence Category",
        loc="center left",
        bbox_to_anchor=(1.02, 0.5),
        frameon=False,
        fontsize=20,
        title_fontsize=20
    )

    plt.subplots_adjust(right=0.85)
    plt.savefig(save_path, bbox_inches="tight", facecolor="white")
    plt.show()

# =============================================================================
# 5. Load, Recode, and Plot
# =============================================================================

# -- 5a. Nonfiltered BF log ROI --
bf_ROI_image = nib.load(bf_ROI_path)
recoded_ROI_image = recode_bayes_factors(bf_ROI_image)
plot_brain_slices(
    nifti_img=recoded_ROI_image,
    title="Bayes Factor ROI Level",
    cmap=cmap,
    save_path=output_nonfiltered
)

